In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
       # print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import os
import cv2
import glob

import shutil 
newpath = '/kaggle/working/images' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
print(os.listdir("../input/dogs-vs-cats/Data/Train/"))

# **Load the data and preprocess it**

In [ ]:
# copy dog training images to working directory
mypath = "../input/dogs-vs-cats/Data/Train/Dogs"
for root, dirs, files in os.walk(mypath):
    for file in files:
      src = mypath+"/"+file
      dst = "/kaggle/working/images"
      shutil.copy(src, dst)

In [ ]:
# copy cat training images to working directory
mypath = "../input/dogs-vs-cats/Data/Train/Cats"
for root, dirs, files in os.walk(mypath):
    for file in files:
      src = mypath+"/"+file
      dst = "/kaggle/working/images"
      shutil.copy(src, dst)

In [ ]:
# copy  testing images to working directory
mypath = "../input/dogs-vs-cats/Data/Test/"
for root, dirs, files in os.walk(mypath):
    for file in files:
      src = mypath+"/"+file
      dst = "/kaggle/working/images"
      shutil.copy(src, dst)

In [ ]:
train_df = pd.read_csv('../input/dogs-vs-cats/Data/Train.csv')
test_df = pd.read_csv('../input/dogs-vs-cats/Data/Sample_Submission.csv')["File_Name"].to_frame()

train_df.head()

In [ ]:
from keras.preprocessing.image import img_to_array

train_data = np.empty((train_df.shape[0], 200, 200, 3), dtype=np.uint8)
for i, fpath in enumerate(train_df["File_Name"].values):
    img = cv2.imread("/kaggle/working/images/"+fpath)
    img=cv2.resize(img,(200,200))
    train_data[i, ...] = img

test_data = np.empty((test_df.shape[0], 200, 200,3), dtype=np.uint8)
for i, fpath in enumerate(test_df["File_Name"].values):
    img = cv2.imread("/kaggle/working/images/"+fpath)
    img=cv2.resize(img,(200,200))
    test_data[i, ...] = img


In [ ]:
print(train_data.shape)

# **Create and train the model**

In [ ]:
import tensorflow as tf
from keras import applications
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.models import Sequential, Model 

    
model= applications.ResNet50(weights = "imagenet", include_top=False, input_shape = (200, 200, 3))
for layer in model.layers:
    layer.trainable = False

#Adding custom Layers 

x = model.layers[-1].output
x = Flatten()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(1000, activation="relu")(x)

predictions = Dense(1, activation="sigmoid")(x)

# creating the final model 
model_final = Model( inputs = model.input,outputs = predictions)
    #model_final.summary()
y_train = np.asarray(train_df["Category"]).astype('float32').reshape((-1,1))
model_final.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
model_final.fit(train_data,y_train,epochs=6,validation_split=0.1)


# **Save the model**

In [ ]:
!mkdir -p saved_model
model_final.save('saved_model/my_model')

# **Convert the model to tflite(for deployment)**

In [ ]:
tflite_model_file = "/kaggle/working/final_model.tflite"
converter = tf.lite.TFLiteConverter.from_saved_model('/kaggle/working/saved_model/my_model')
#converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

# **Generating output Dog:1 Cat:0**

In [ ]:
ans=model_final.predict(test_data)

In [ ]:
ans= ans>0.5
ans

In [ ]:
ans=ans*1
ans

In [ ]:
output=pd.DataFrame()
output['File_Name']=test_df['File_Name']
output['Category']=ans
output.to_csv('mobilenetv2_5_adam.csv',index=False)

In [ ]:
tf.__version__